In [1]:
import pandas as pd
import torch
from torch.utils.data import DataLoader, TensorDataset

# Read the dataset
data = pd.read_csv("data.csv")

# Split features and labels
x = data[['num1', 'num2']].values
y = data['label'].values

# Convert to PyTorch tensors
x = torch.tensor(x, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

# Create a TensorDataset
dataset = TensorDataset(x, y)

# Define batch size
batch_size = 1

# Create a DataLoader
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [10]:
import torch
import torch.nn as nn

import torch.nn as nn

class TwinRNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(TwinRNN, self).__init__()
        self.embedTwin = nn.Embedding(100, 6)
        self.lstmTwin = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        # Reshape input to have a third dimension
        x = x.unsqueeze(2)

        # Twin 1
        xTwin1 = x[:, :, 0]  # Select first feature
        embedsTwin1 = self.embedTwin(xTwin1.long())
        inputsTwin1 = torch.cat((embedsTwin1.float(), xTwin1.float()), 2)
        lstm_out1, (h_n1, c_n1) = self.lstmTwin(inputsTwin1)
        featMapTwin1 = h_n1[-1, :, :]

        # Twin 2
        xTwin2 = x[:, :, 1]  # Select second feature
        embedsTwin2 = self.embedTwin(xTwin2.long())
        inputsTwin2 = torch.cat((embedsTwin2.float(), xTwin2.float()), 2)
        lstm_out2, (h_n2, c_n2) = self.lstmTwin(inputsTwin2)
        featMapTwin2 = h_n2[-1, :, :]

        # Combine twins
        featMap = torch.cat((featMapTwin1, featMapTwin2), dim=1)

        # Fully connected layer
        output = self.fc(featMap)
        return output


# Example usage
input_size = 100  # Define your input size according to your dataset
hidden_size = 64
num_layers = 2
output_size = 2  # Define your output size

model = TwinRNN(input_size, hidden_size, num_layers, output_size)

# Assuming you have your dataset loaded as tensors x and y
# x shape: (batch_size, sequence_length, input_size)
# y shape: (batch_size, output_size)

# Define your loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)



In [11]:
# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    # Forward pass
    outputs = model(x)
    loss = criterion(outputs, y)
    
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    # Print loss every epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

IndexError: index out of range in self